In [1]:
from django.db import models
from cast.filters import PostFilterset

In [2]:
PostTag.objects.all()

<QuerySet [<PostTag: Weeknotes 2023-07-31 tagged with images>, <PostTag: Weeknotes 2023-08-21 tagged with django>, <PostTag: Weeknotes 2023-08-21 tagged with rust>, <PostTag: Claas im Monkeyspot tagged with video>, <PostTag: Full-Stack Team Now Open for Projects: A First-Time Announcement! 🚀 tagged with freelancing>]>

In [3]:
tag = PostTag.objects.first()

In [4]:
blog = Blog.objects.get(slug="ephes_blog")

In [5]:
blog.unfiltered_published_posts.filter(tags__name="foobar")

<PageQuerySet []>

In [48]:
queryset = blog.unfiltered_published_posts.filter(pk__lt=100)
queryset = blog.unfiltered_published_posts

In [22]:
category_count_queryset = PostCategory.objects.annotate(
    num_posts=models.Count("post", filter=models.Q(post__in=queryset))
)
category_counts = {}
for category in category_count_queryset:
    category_counts[category.slug] = (category.name, category.num_posts)  # type: ignore
print(category_counts)

{'weeknotes': ('WeekNotes', 2), 'fotos': ('Fotos', 1), 'til': ('Today I Learned', 1), 'video': ('Video', 0)}


In [24]:
tag_count_queryset = PostTag.objects.annotate(
    num_posts=models.Count("content_object", filter=models.Q(content_object__in=queryset))
)
tag_counts = {}
for tag in tag_count_queryset:
    tag_counts[tag.tag.slug] = (tag.tag.name, tag.num_posts)  # type: ignore
print("set tag counts: ", tag_counts)

set tag counts:  {'claas': ('claas', 1), 'foobar': ('foobar', 1)}


In [77]:
tag_count_queryset = PostTag.objects.annotate(
    num_posts=models.Count("tag__slug")
).annotate(slug=models.F("tag__slug"))
tag_counts = {}
for tag in tag_count_queryset:
    print("slug: ", tag.slug)
#    tag_counts[tag.tag.slug] = (tag.tag.name, tag.num_posts)  # type: ignore
# print("set tag counts: ", tag_counts)

slug:  foobar
slug:  foobar
slug:  foobar
slug:  claas


In [84]:
PostTag.objects.annotate(num_posts=models.Count("content_object")).aggregate(slug=models.F("tag__slug"))

TypeError: slug is not an aggregate expression

In [86]:
pt_qs = PostTag.objects.annotate(
    num_posts=models.Count("content_object", distinct=True)
)
print(pt_qs.query)
print(pt_qs.values())

SELECT "cast_posttag"."id", "cast_posttag"."tag_id", "cast_posttag"."content_object_id", COUNT(DISTINCT "cast_posttag"."content_object_id") AS "num_posts" FROM "cast_posttag" GROUP BY "cast_posttag"."id"
<QuerySet [{'id': 2, 'tag_id': 1, 'content_object_id': 392, 'num_posts': 1}, {'id': 3, 'tag_id': 109, 'content_object_id': 392, 'num_posts': 1}, {'id': 5, 'tag_id': 109, 'content_object_id': 400, 'num_posts': 1}, {'id': 6, 'tag_id': 109, 'content_object_id': 402, 'num_posts': 1}]>


In [71]:
%%time
tag_count_queryset = Tag.objects.annotate(
    num_posts=Count("post", filter=models.Q(post__in=queryset))
).filter(num_posts__gt=0)
tag_counts = {}
for tag in tag_count_queryset:
    tag_counts[tag.slug] = (tag.name, tag.num_posts)  # type: ignore
print("set tag counts: ", tag_counts)

set tag counts:  {'foobar': ('foobar', 3), 'claas': ('claas', 1)}
CPU times: user 5.05 ms, sys: 2.16 ms, total: 7.21 ms
Wall time: 6.94 ms


In [72]:
%%time
{slug: (name, num_posts) for slug, name, num_posts in Tag.objects.annotate(
    num_posts=Count("post", filter=models.Q(post__in=queryset))
).filter(num_posts__gt=0).values_list("slug", "name", "num_posts")}

CPU times: user 4.7 ms, sys: 2.14 ms, total: 6.84 ms
Wall time: 6.82 ms


{'foobar': ('foobar', 3), 'claas': ('claas', 1)}

In [16]:
tag = PostTag.objects.first()

In [30]:
tag.tag

<Tag: foobar>

In [21]:
blog.unfiltered_published_posts.filter(tags__name__in=["foobar"])

<PageQuerySet []>

In [22]:
blog.unfiltered_published_posts.filter(tags__name="foobar")

<PageQuerySet []>

In [23]:
post = Post.objects.get(pk=392)

In [32]:
tag = post.tags.first()

In [34]:
blog.unfiltered_published_posts.filter(tags__in=[tag])

<PageQuerySet []>

In [33]:
tag

<Tag: claas>

In [35]:
Post.objects.filter(tags__in=[tag])

<PageQuerySet [<Post: Weeknotes 2023-07-17>]>

In [39]:
Post.objects.filter(tags__name="foobar")

<PageQuerySet [<Post: Weeknotes 2023-07-17>]>

In [40]:
blog.unfiltered_published_posts.filter(tags__name="foobar")

<PageQuerySet []>

In [37]:
Post.objects.all()

<PageQuerySet [<Post: Zu ersten Mal etwas Möhrenbrei>, <Post: Zu Besuch bei den Grosseltern>, <Post: Tapas mit Heike und Chrissie>, <Post: Martin und Marcus kommen zu Besuch>, <Post: Kaffeetrinken in der Spoerl-Fabrik>, <Post: In alten Möbeln>, <Post: Die U5 ist überstanden>, <Post: Spieldecke>, <Post: Schnullerversuch>, <Post: Ein Wochenende in Abentheuer>, <Post: Sofagebrabbel>, <Post: Claas zieht sich zum ersten Mal am Sofa hoch>, <Post: Zum ersten Mal Draussen>, <Post: Kinderwagen ist ok>, <Post: Erste Stehversuche>, <Post: Kurzbesuch bei Uroma Thea>, <Post: Zu Besuch in Meersen bei Onkel Jo>, <Post: Weihnachten 2017>, <Post: Mit Katharina bei Sulis>, <Post: Claas hält sich zum ersten Mal alleine am Tisch fest>, '...(remaining elements truncated)...']>

In [42]:
Post.objects.live().filter(tags__name="foobar")

<PageQuerySet [<Post: Weeknotes 2023-07-17>]>

In [43]:
blog

<Blog: Claas und Nora sagen Hallo>

In [44]:
PostTag.objects.all()

<QuerySet [<PostTag: Weeknotes 2023-07-17 tagged with claas>, <PostTag: Weeknotes 2023-07-17 tagged with foobar>]>

In [46]:
tag = PostTag.objects.first()

In [49]:
tag.tagged_items()

AttributeError: 'PostTag' object has no attribute 'tagged_items'

In [9]:
tag_count_queryset = PostTag.objects.annotate(
    num_posts=models.Count(
        "content_object",
        filter=models.Q(content_object__in=blog.unfiltered_published_posts)
    )
)

In [10]:
tag_count_queryset

<QuerySet [<PostTag: Weeknotes 2023-07-17 tagged with claas>, <PostTag: Weeknotes 2023-07-17 tagged with foobar>]>

In [12]:
tag_counts = {}
for tag in tag_count_queryset:
    tag_counts[tag.tag.name] = tag.num_posts  # type: ignore

In [13]:
tag_counts

{'claas': 1, 'foobar': 1}

In [14]:
set(PostTag.objects.values_list("tag__name", flat=True))

{'claas', 'foobar'}

In [6]:
post_tag = PostTag.objects.first()

In [8]:
post_tag.tag.slug

'claas'

In [3]:
set(PostCategory.objects.values_list("slug", flat=True))

{'fotos', 'til', 'video', 'weeknotes'}

# Facet Counts are wrong (categories instead of tags)

In [1]:
from django.db import models
from cast.filters import PostFilterset

In [2]:
blog = Blog.objects.get(slug="ephes_blog")
queryset = blog.unfiltered_published_posts

In [3]:
filterset = PostFilterset({}, queryset, fetch_facet_counts=True)

facet count choices:  []
has facets with posts in field:  False
facet count choices:  []
has facets with posts in field:  False
facet count choices:  [('fotos', 'Fotos (1)'), ('til', 'Today I Learned (1)'), ('weeknotes', 'WeekNotes (2)')]
has facets with posts in field:  True
set tag counts:  {'claas': ('claas', 1), 'foobar': ('foobar', 1)}
facet count choices:  [('claas', 'claas (1)'), ('foobar', 'foobar (1)')]
has facets with posts in field:  True
facet counts:  {'claas': ('claas', 1), 'foobar': ('foobar', 1)}


In [4]:
filterset.filters["tag_facets"].facet_counts

{'claas': ('claas', 1), 'foobar': ('foobar', 1)}

In [5]:
tags_field = filterset.form.fields["tag_facets"]

In [6]:
list(tags_field.choices)

[('', '---------'), ('claas', 'claas (1)'), ('foobar', 'foobar (1)')]

In [7]:
categories_field = filterset.form.fields["category_facets"]

In [8]:
list(categories_field.choices)

[('', '---------'),
 ('fotos', 'Fotos (1)'),
 ('til', 'Today I Learned (1)'),
 ('weeknotes', 'WeekNotes (2)')]

In [9]:
filterset._meta.fields

['search', 'date', 'date_facets', 'category_facets', 'tag_facets']

In [10]:
filterset.filters

OrderedDict([('search', <django_filters.filters.CharFilter at 0x10e1c7250>),
             ('date',
              <django_filters.filters.DateFromToRangeFilter at 0x10e1ba1d0>),
             ('category_facets',
              <cast.filters.CategoryFacetFilter at 0x107d02c10>),
             ('tag_facets', <cast.filters.TagFacetFilter at 0x10de6b350>),
             ('date_facets', <cast.filters.DateFacetFilter at 0x10e1b0cd0>)])

In [18]:
filterset._form

<PostFiltersetForm bound=True, valid=True, fields=(search;date;category_facets;tag_facets;date_facets)>

In [9]:
form = filterset.form
list(form.fields["tag_facets"].choices)

[('', '---------'), ('claas', 'claas (1)'), ('foobar', 'foobar (1)')]

In [10]:
form["tag_facets"]

In [50]:
bound = dict(form._bound_items())
tag_bound = bound["tag_facets"]

In [58]:
tag_bound.field

In [61]:
dir(tag_bound)

['__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__html__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_has_changed',
 'as_hidden',
 'as_text',
 'as_textarea',
 'as_widget',
 'auto_id',
 'build_widget_attrs',
 'css_classes',
 'data',
 'errors',
 'field',
 'form',
 'help_text',
 'html_initial_id',
 'html_initial_name',
 'html_name',
 'id_for_label',
 'initial',
 'is_hidden',
 'label',
 'label_tag',
 'legend_tag',
 'name',
 'subwidgets',
 'use_fieldset',
 'value',
 'widget_type']

In [69]:
tag_bound.initial

In [52]:
str(tag_bound)

'<div class="cast-date-facet-container" id="id_tag_facets">\n<div class="cast-date-facet-item"><a class="selected" href="?tag_facets=">All</a><input type="hidden" name="tag_facets" value=""></div>\n</div>'

In [39]:
for field, _ in form.get_context()["fields"]:
    print(field)

<input type="text" name="search" id="id_search">
<input type="date" name="date_after" placeholder="YYYY/MM/DD" id="id_date_0">

-<input type="date" name="date_before" placeholder="YYYY/MM/DD" id="id_date_1">
<div class="cast-date-facet-container" id="id_category_facets">
<div class="cast-date-facet-item"><a class="selected" href="?category_facets=">All</a><input type="hidden" name="category_facets" value=""></div>
</div>
<div class="cast-date-facet-container" id="id_tag_facets">
<div class="cast-date-facet-item"><a class="selected" href="?tag_facets=">All</a><input type="hidden" name="tag_facets" value=""></div>
</div>
<div class="cast-date-facet-container" id="id_date_facets">
<div class="cast-date-facet-item"><a class="selected" href="?date_facets=">All</a><input type="hidden" name="date_facets" value=""></div>
</div>


In [24]:
form.render()

/var/folders/35/mhty_0hd42ddfvyk3_09f1hc0000gn/T/ipykernel_91043/307509643.py:1: RemovedInDjango50Warning: The "default.html" templates for forms and formsets will be removed. These were proxies to the equivalent "table.html" templates, but the new "div.html" templates will be the default from Django 5.0. Transitional renderers are provided to allow you to opt-in to the new output style now. See https://docs.djangoproject.com/en/4.2/releases/4.1/ for more details
  form.render()


'<tr>\n    <th><label for="id_search">Search:</label></th>\n    <td>\n      \n      <input type="text" name="search" id="id_search">\n      \n      \n    </td>\n  </tr>\n\n  <tr>\n    <th><label>Date:</label></th>\n    <td>\n      \n      <input type="date" name="date_after" placeholder="YYYY/MM/DD" id="id_date_0">\n\n-<input type="date" name="date_before" placeholder="YYYY/MM/DD" id="id_date_1">\n      \n      \n    </td>\n  </tr>\n\n  <tr>\n    <th><label for="id_category_facets">Categories:</label></th>\n    <td>\n      \n      <div class="cast-date-facet-container" id="id_category_facets">\n<div class="cast-date-facet-item"><a class="selected" href="?category_facets=">All</a><input type="hidden" name="category_facets" value=""></div>\n</div>\n      \n      \n    </td>\n  </tr>\n\n  <tr>\n    <th><label for="id_tag_facets">Tags:</label></th>\n    <td>\n      \n      <div class="cast-date-facet-container" id="id_tag_facets">\n<div class="cast-date-facet-item"><a class="selected" href

In [13]:
Form = filterset.get_form_class()

facet count choices:  [('fotos', 'Fotos (1)'), ('til', 'Today I Learned (1)'), ('weeknotes', 'WeekNotes (2)')]
has facets with posts in field:  True
facet count choices:  [('claas', 'claas (1)'), ('foobar', 'foobar (1)')]
has facets with posts in field:  True


In [14]:
form = Form({})

In [15]:
list(form.fields["tag_facets"].choices)

[('', '---------'), ('claas', 'claas (1)'), ('foobar', 'foobar (1)')]

In [16]:
form.fields["date_facets"]

In [17]:
form.fields["tag_facets"]

In [9]:
dir(form)

NameError: name 'form' is not defined